In [144]:
import pandas as pd
import json
df = pd.read_excel('./data/dataset.xlsx')

In [145]:
df.head()

,Patient ID,Patient age quantile,SARS-Cov-2 exam result,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,...,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
0,44477f75e8169d2,13,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,126e9dd13932f68,17,negative,0,0,0,0.236515,-0.02234,-0.517413,0.010677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a46b4402a0e5696,8,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,f7d619a94f97c45,5,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,d9e41465789c2b5,15,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
df[["Urine - Leukocytes"]] = df[["Urine - Leukocytes"]].replace('<1000', '0')
df['Urine - Leukocytes'] = df['Urine - Leukocytes'].astype(float)

In [147]:
df["Urine - pH"][177] = float('NaN')
df['Urine - pH'] = df['Urine - pH'].astype(float)
df["Patient age quantile"] = df["Patient age quantile"].astype(float)

In [148]:
df1 = df.drop(['Mycoplasma pneumoniae', 'Urine - Nitrite', 'Urine - Sugar', 'Partial thromboplastin time\xa0(PTT)\xa0', 'Prothrombin time (PT), Activity', 'D-Dimer', 'Fio2 (venous blood gas analysis)'], axis=1)

In [149]:
def getAttrInfo(df):
    attrInfo = {}
    for name in list(df.columns):
        num = float(len(df[name].unique()))
        varType = 'continuous' if num > 6 else 'categorical'
        attrInfo[name] = {'numDistinctValues': num, 'type': varType, 'numNull': df[name].isna().sum().item()}
        if (varType == 'categorical'):
            attrInfo[name]['values'] = list(df[name].unique())

    attrInfo["Patient ID"]["type"] = "categorical"
    attrInfo["Patient ID"]["values"] = list(df['Patient ID'].unique())
    
    return attrInfo

In [150]:
attrInfo = getAttrInfo(df1)

In [151]:
hasToBeEncoded = {}
df1_encoded = df1.copy()

In [152]:
df1_encoded["Urine - Crystals"].unique()

array([nan, 'Ausentes', 'Urato Amorfo --+', 'Oxalato de Cálcio +++',
       'Oxalato de Cálcio -++', 'Urato Amorfo +++'], dtype=object)

In [153]:
for item in attrInfo:
    if (attrInfo[item]['type'] == 'categorical'):
        
        if (attrInfo[item]['values'] == [0,1]):
            print(attrInfo[item])
            df1_encoded[item] = df1_encoded[item].replace([0,1], ['no', 'yes'])
    else:
        print(item)
        attrInfo[item]['min'] = df1_encoded[item].min()
        attrInfo[item]['max'] = df1_encoded[item].max()
df1_encoded.head()

Patient age quantile
{'numDistinctValues': 2.0, 'type': 'categorical', 'numNull': 0, 'values': [0, 1]}
{'numDistinctValues': 2.0, 'type': 'categorical', 'numNull': 0, 'values': [0, 1]}
{'numDistinctValues': 2.0, 'type': 'categorical', 'numNull': 0, 'values': [0, 1]}
Hematocrit
Hemoglobin
Platelets
Mean platelet volume 
Red blood Cells
Lymphocytes
Mean corpuscular hemoglobin concentration (MCHC)
Leukocytes
Basophils
Mean corpuscular hemoglobin (MCH)
Eosinophils
Mean corpuscular volume (MCV)
Monocytes
Red blood cell distribution width (RDW)
Serum Glucose
Neutrophils
Urea
Proteina C reativa mg/dL
Creatinine
Potassium
Sodium
Alanine transaminase
Aspartate transaminase
Gamma-glutamyltransferase 
Total Bilirubin
Direct Bilirubin
Indirect Bilirubin
Alkaline phosphatase
Ionized calcium 
Magnesium
pCO2 (venous blood gas analysis)
Hb saturation (venous blood gas analysis)
Base excess (venous blood gas analysis)
pO2 (venous blood gas analysis)
Total CO2 (venous blood gas analysis)
pH (venous bloo

,Patient ID,Patient age quantile,SARS-Cov-2 exam result,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,...,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
0,44477f75e8169d2,13.0,negative,no,no,no,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,126e9dd13932f68,17.0,negative,no,no,no,0.236515,-0.02234,-0.517413,0.010677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a46b4402a0e5696,8.0,negative,no,no,no,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,f7d619a94f97c45,5.0,negative,no,no,no,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,d9e41465789c2b5,15.0,negative,no,no,no,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [154]:
df1_enc_renamed = df1_encoded.rename({'Patient addmited to regular ward (1=yes, 0=no)': 'Patient addmited to regular ward', 'Patient addmited to semi-intensive unit (1=yes, 0=no)': 'Patient addmited to semi-intensive unit', 'Patient addmited to intensive care unit (1=yes, 0=no)': 'Patient addmited to intensive care unit'}, axis=1)

In [155]:
df1_enc_renamed.to_json('./data/data.json', orient='records')

In [126]:
import simplejson

In [156]:
with open('./data/attrInfo.json', 'w') as ai:
    json.dump(attrInfo, ai, allow_nan=False)

TypeError: Object of type int64 is not JSON serializable